## <b><font color='darkblue'>Events</font></b>
([source](https://google.github.io/adk-docs/events/)) <font size='3ptx'><b>Events are the fundamental units of information flow within the [Agent Development Kit](https://google.github.io/adk-docs/) (ADK).</b> They represent every significant occurrence during an agent's interaction lifecycle, from initial user input to the final response and all the steps in between. Understanding events is crucial because they are the primary way components communicate, state is managed, and control flow is directed.</font>

### <b><font color='darkgreen'>What Events Are and Why They Matter</font></b>
([source](https://google.github.io/adk-docs/events/#what-events-are-and-why-they-matter)) <font size='3ptx'><b>An `Event` in ADK is an immutable record representing a specific point in the agent's execution.</b> It captures user messages, agent replies, requests to use tools (function calls), tool results, state changes, control signals, and errors.</font>

Technically, it's an instance of the <b><font color='blue'>google.adk.events.Event</font></b> class, which builds upon the basic `LlmResponse` structure by adding essential ADK-specific metadata and an `actions` payload. The class signature looks like:
```python
# Conceptual Structure of an Event (Python)
# from google.adk.events import Event, EventActions
# from google.genai import types

# class Event(LlmResponse): # Simplified view
#     # --- LlmResponse fields ---
#     content: Optional[types.Content]
#     partial: Optional[bool]
#     # ... other response fields ...

#     # --- ADK specific additions ---
#     author: str          # 'user' or agent name
#     invocation_id: str   # ID for the whole interaction run
#     id: str              # Unique ID for this specific event
#     timestamp: float     # Creation time
#     actions: EventActions # Important for side-effects & control
#     branch: Optional[str] # Hierarchy path
#     # ...
```

`Events` are central to ADK's operation for several key reasons:
* <b><font size='3ptx'>Communication</font></b>: They serve as the standard message format between the user interface, the `Runner`, agents, the LLM, and tools. Everything flows as an `Event`.
* <b><font size='3ptx'>Signaling State & Artifact Changes</font></b>: Events carry instructions for state modifications and track artifact updates. <b>The `SessionService` uses these signals to ensure persistence</b>. In Python changes are signaled via `event.actions.state_delta` and `event.actions.artifact_delta`.
* <b><font size='3ptx'>Control Flow</font></b>: Specific fields like `event.actions.transfer_to_agent` or `event.actions.escalate` act as signals that direct the framework, determining which agent runs next or if a loop should terminate.
* <b><font size='3ptx'>History & Observability</font></b>: The sequence of events recorded in `session.events` provides <b>a complete, chronological history of an interaction, invaluable for debugging, auditing, and understanding agent behavior step-by-step</b>.

In essence, the entire process, from a user's query to the agent's final answer, is orchestrated through the generation, interpretation, and processing of `Event` objects.

### <b><font color='darkgreen'>Understanding and Using Events</font></b>
([source](https://google.github.io/adk-docs/events/#understanding-and-using-events)) <font size='3ptx'><b>As a developer, you'll primarily interact with the stream of events yielded by the `Runner`.</b> Here's how to understand and extract information from them:</font>

#### <b>Identifying Event Origin and Type</b>
Quickly determine what an event represents by checking:
* <b><font size='3ptx'>Who sent it? (`event.author`)</font></b>
    - `'user'`: Indicates input directly from the end-user.
    - `'AgentName'`: Indicates output or action from a specific agent (e.g., `'WeatherAgent'`, `'SummarizerAgent'`).
* <b><font size='3ptx'>What's the main payload? (`event.content` and `event.content.parts`)</font></b>
    - <b>Text</b>: Indicates a conversational message. For Python, check if `event.content.parts[0].text` exists.
    - <b>Tool Call Request</b>: Check `event.get_function_calls()`. If not empty, the LLM is asking to execute one or more tools. Each item in the list has `.name` and `.args`.
    - <b>Tool Result</b>: Check `event.get_function_responses()`. If not empty, this event carries the result(s) from tool execution(s). Each item has `.name` and `.response` (<font color='brown'>the dictionary returned by the tool</font>). Note: For history structuring, the `role` inside the `content` is often `'user'`, but the event `author` is typically the agent that requested the tool call.
* <b><font size='3ptx'>Is it streaming output? (`event.partial`)</font></b> Indicates whether this is an incomplete chunk of text from the LLM.
    - `True`: More text will follow.
    - `False` or `None`: This part of the content is complete (<font color='brown'>though the overall turn might not be finished if `turn_complete` is also false</font>).

Below is the sample code for validation introduced above:
```python
# Pseudocode: Basic event identification (Python)
# async for event in runner.run_async(...):
#     print(f"Event from: {event.author}")
#
#     if event.content and event.content.parts:
#         if event.get_function_calls():
#             print("  Type: Tool Call Request")
#         elif event.get_function_responses():
#             print("  Type: Tool Result")
#         elif event.content.parts[0].text:
#             if event.partial:
#                 print("  Type: Streaming Text Chunk")
#             else:
#                 print("  Type: Complete Text Message")
#         else:
#             print("  Type: Other Content (e.g., code result)")
#     elif event.actions and (event.actions.state_delta or event.actions.artifact_delta):
#         print("  Type: State/Artifact Update")
#     else:
#         print("  Type: Control Signal or Other")
```

#### <b>Extracting Key Information</b>
Once you know the event type, next step is to access the relevant data:
* <b><font size='3ptx'>Text Content</font></b>: Always check for the presence of content and parts before accessing text. In Python its `text = event.content.parts[0].text`.
* <b><font size='3ptx'>Function Call Details</font></b>:

```python
calls = event.get_function_calls()
if calls:
    for call in calls:
        tool_name = call.name
        arguments = call.args # This is usually a dictionary
        print(f"  Tool: {tool_name}, Args: {arguments}")
        # Application might dispatch execution based on this
```

* <b><font size='3ptx'>Function Response Details</font></b>:

```python
responses = event.get_function_responses()
if responses:
    for response in responses:
        tool_name = response.name
        result_dict = response.response # The dictionary returned by the tool
        print(f"  Tool Result: {tool_name} -> {result_dict}")
```

* <b><font size='3ptx'>Identifiers</font></b>:
    - `event.id`: Unique ID for this specific event instance.
    - `event.invocation_id`: ID for the entire user-request-to-final-response cycle this event belongs to. Useful for logging and tracing.

#### <b>Detecting Actions and Side Effects</b>
The `event.actions` object signals changes that occurred or should occur. Always check if `event.actions` and it's fields/ methods exists before accessing them.

* <b><font size='3ptx'>State Changes</font></b>: Gives you a collection of key-value pairs that were modified in the session state during the step that produced this event.

```python
if event.actions and event.actions.state_delta:
    print(f"  State changes: {event.actions.state_delta}")
    # Update local UI or application state if necessary
```

* <b><font size='3ptx'>Artifact Saves</font></b>: Gives you a collection indicating which artifacts were saved and their new version number (<font color='brown'>or relevant `Part` information</font>).

```python
if event.actions and event.actions.artifact_delta:
    print(f"  Artifacts saved: {event.actions.artifact_delta}")
    # UI might refresh an artifact list
```

* <b><font size='3ptx'>Control Flow Signals</font></b>: Check boolean flags or string values:
    - <b>event.actions.transfer_to_agent (`string`)</b>: Control should pass to the named agent.
    - <b>event.actions.escalate (`bool`)</b>: A loop should terminate.
    - <b>event.actions.skip_summarization (`bool`)</b>: A tool result should not be summarized by the LLM.

```python
if event.actions:
    if event.actions.transfer_to_agent:
        print(f"  Signal: Transfer to {event.actions.transfer_to_agent}")
    if event.actions.escalate:
        print("  Signal: Escalate (terminate loop)")
    if event.actions.skip_summarization:
        print("  Signal: Skip summarization for tool result")
```

#### <b>Determining if an `Event` is a "Final" Response</b>
Use the built-in helper method `event.is_final_response()` to identify events suitable for display as the agent's complete output for a turn.
* <b><font size='3ptx'>Purpose</font></b>: Filters out intermediate steps (<font color='brown'>like tool calls, partial streaming text, internal state updates</font>) from the final user-facing message(s).
* <b><font size='3ptx'>When `True`?</font></b>
    - The event contains a tool result (`function_response`) and `skip_summarization` is `True`.
    - The event contains a tool call (`function_call`) for a tool marked as `is_long_running=True`.
        - `event.longRunningToolIds().isPresent() && !event.longRunningToolIds().get().isEmpty() is true`.
    - OR, all of the following are met:
        - No function calls (`get_function_calls()` is empty).
        - No function responses (`get_function_responses()` is empty).
        - Not a partial stream chunk (`partial` is not `True`).
        - Doesn't end with a code execution result that might need further processing/display.
* <b><font size='3ptx'>Usage</font></b>: Filter the event stream in your application logic.

```python
# Pseudocode: Handling final responses in application (Python)
# full_response_text = ""
# async for event in runner.run_async(...):
#     # Accumulate streaming text if needed...
#     if event.partial and event.content and event.content.parts and event.content.parts[0].text:
#         full_response_text += event.content.parts[0].text
#
#     # Check if it's a final, displayable event
#     if event.is_final_response():
#         print("\n--- Final Output Detected ---")
#         if event.content and event.content.parts and event.content.parts[0].text:
#              # If it's the final part of a stream, use accumulated text
#              final_text = full_response_text + (event.content.parts[0].text if not event.partial else "")
#              print(f"Display to user: {final_text.strip()}")
#              full_response_text = "" # Reset accumulator
#         elif event.actions and event.actions.skip_summarization and event.get_function_responses():
#              # Handle displaying the raw tool result if needed
#              response_data = event.get_function_responses()[0].response
#              print(f"Display raw tool result: {response_data}")
#         elif hasattr(event, 'long_running_tool_ids') and event.long_running_tool_ids:
#              print("Display message: Tool is running in background...")
#         else:
#              # Handle other types of final responses if applicable
#              print("Display: Final non-textual response or signal.")
```

By carefully examining these aspects of an event, you can build robust applications that react appropriately to the rich information flowing through the ADK system.

### <b><font color='darkgreen'>How Events Flow: Generation and Processing</font></b>
([source](https://google.github.io/adk-docs/events/#how-events-flow-generation-and-processing)) <font size='3ptx'><b>Events are created at different points and processed systematically by the framework.</b> Understanding this flow helps clarify how actions and history are managed.</font>

#### <b>Generation Sources:</b>
* <b><font size='3ptx'>User Input</font></b>: The `Runner` typically wraps initial user messages or mid-conversation inputs into an `Event` with `author='user'`.
* <b><font size='3ptx'>Agent Logic</font></b>: Agents (`BaseAgent`, `LlmAgent`) explicitly `yield Event(...)` objects (<font color='brown'>setting `author=self.name`</font>) to communicate responses or signal actions.
* <b><font size='3ptx'>LLM Responses</font></b>: The ADK model integration layer translates raw LLM output (<font color='brown'>text, function calls, errors</font>) into `Event` objects, authored by the calling agent.
* <b><font size='3ptx'>Tool Results</font></b>: After a tool executes, the framework generates an `Event` containing the `function_response`. The author is typically the agent that requested the tool, while the role inside the content is set to `'user'` for the LLM history.

#### <b>Processing Flow</b>
* <b><font size='3ptx'>Yield/Return</font></b>: An event is generated and yielded by its source.
* <b><font size='3ptx'>`Runner` Receives</font></b>: The main `Runner` executing the agent receives the event.
* <b><font size='3ptx'>`SessionService` Processing</font></b>: The `Runner` sends the event to the configured `SessionService`. This is a critical step:
    - <b>Applies Deltas</b>: The service merges `event.actions.state_delta` into `session.state` and updates internal records based on `event.actions.artifact_delta`. (<font color='brown'>Note: The actual artifact saving usually happened earlier when `context.save_artifact` was called</font>).
    - <b>Finalizes Metadata</b>: Assigns a unique `event.id` if not present, may update `event.timestamp`.
    - <b>Persists to History</b>: Appends the processed event to the `session.events` list.
* <b><font size='3ptx'>External Yield</font></b>: The `Runner` yields the processed event outwards to the calling application (<font color='brown'>e.g., the code that invoked `runner.run_async`</font>).

This flow ensures that state changes and history are consistently recorded alongside the communication content of each event.

### <b><font color='darkgreen'>Common Event Examples (Illustrative Patterns)</font></b>
([source](https://google.github.io/adk-docs/events/#common-event-examples-illustrative-patterns)) Here are concise examples of typical events you might see in the stream:

#### <b>User Input</b>:
```python
{
  "author": "user",
  "invocation_id": "e-xyz...",
  "content": {"parts": [{"text": "Book a flight to London for next Tuesday"}]}
  // actions usually empty
}
```

#### <b>Agent Final Text Response: (`is_final_response() == True`)</b>
```python
{
  "author": "TravelAgent",
  "invocation_id": "e-xyz...",
  "content": {"parts": [{"text": "Okay, I can help with that. Could you confirm the departure city?"}]},
  "partial": false,
  "turn_complete": true
  // actions might have state delta, etc.
}
```

#### <b>Agent Streaming Text Response: (`is_final_response() == False`)</b>
```python
{
  "author": "SummaryAgent",
  "invocation_id": "e-abc...",
  "content": {"parts": [{"text": "The document discusses three main points:"}]},
  "partial": true,
  "turn_complete": false
}
// ... more partial=True events follow ...
```

#### <b>Tool Call Request (by LLM): (`is_final_response() == False`)</b>
```python
{
  "author": "TravelAgent",
  "invocation_id": "e-xyz...",
  "content": {"parts": [{"function_call": {"name": "find_airports", "args": {"city": "London"}}}]}
  // actions usually empty
}
```

#### <b>Tool Result Provided (to LLM): (`is_final_response() depends on skip_summarization`)</b>

```python
{
  "author": "TravelAgent", // Author is agent that requested the call
  "invocation_id": "e-xyz...",
  "content": {
    "role": "user", // Role for LLM history
    "parts": [{"function_response": {"name": "find_airports", "response": {"result": ["LHR", "LGW", "STN"]}}}]
  }
  // actions might have skip_summarization=True
}
```

#### <b>State/Artifact Update Only: (`is_final_response() == False`)</b>
```python
{
  "author": "InternalUpdater",
  "invocation_id": "e-def...",
  "content": null,
  "actions": {
    "state_delta": {"user_status": "verified"},
    "artifact_delta": {"verification_doc.pdf": 2}
  }
}
```

#### <b>Agent Transfer Signal: (`is_final_response() == False`)</b>
```python
{
  "author": "OrchestratorAgent",
  "invocation_id": "e-789...",
  "content": {"parts": [{"function_call": {"name": "transfer_to_agent", "args": {"agent_name": "BillingAgent"}}}]},
  "actions": {"transfer_to_agent": "BillingAgent"} // Added by framework
}
```

#### <b>Loop Escalation Signal: (`is_final_response() == False`)</b>
```python
{
  "author": "CheckerAgent",
  "invocation_id": "e-loop...",
  "content": {"parts": [{"text": "Maximum retries reached."}]}, // Optional content
  "actions": {"escalate": true}
}
```

### <b><font color='darkgreen'>Additional Context and Event Details</font></b>
([source](https://google.github.io/adk-docs/events/#additional-context-and-event-details)) <font size='3ptx'><b>Beyond the core concepts, here are a few specific details about context and events that are important for certain use cases:</b></font>

#### <b>1. `ToolContext.function_call_id` (Linking Tool Actions):</b>
* When an LLM requests a tool (`FunctionCall`), that request has an ID. The `ToolContext` provided to your tool function includes this `function_call_id`.
* <b><font size='3ptx'>Importance</font></b>: This ID is crucial for linking actions like authentication back to the specific tool request that initiated them, especially if multiple tools are called in one turn. The framework uses this ID internally.

#### <b>2. How State/Artifact Changes are Recorded:</b>
* When you modify state or save an artifact using `CallbackContext` or `ToolContext`, these changes aren't immediately written to persistent storage.
* Instead, they populate the `state_delta` and `artifact_delta` fields within the <b><font color='blue'>EventActions</font></b> object.
* This <b><font color='blue'>EventActions</font></b> object is attached to the next event generated after the change (<font color='brown'>e.g., the agent's response or a tool result event</font>).
* The `SessionService.append_event` method reads these deltas from the incoming event and applies them to the session's persistent state and artifact records. This ensures changes are tied chronologically to the event stream.

#### <b>3. State Scope Prefixes (`app:`, `user:`, `temp:`):</b>
* When managing state via `context.state`, you can optionally use prefixes
    - `app:my_setting`: Suggests state relevant to the entire application (<font color='brown'>requires a persistent `SessionService`</font>).
    - `user:user_preference`: Suggests state relevant to the specific user across sessions (<font color='brown'>requires a persistent `SessionService`</font>).
    - `temp:intermediate_result` or no prefix: Typically session-specific or temporary state for the current invocation.
* The underlying `SessionService` determines how these prefixes are handled for persistence.

#### <b>4. Error Events:</b>
* An `Event` can represent an error. Check the `event.error_code` and `event.error_message` fields (<font color='brown'>inherited from `LlmResponse`</font>).
* Errors might originate from the LLM (<font color='brown'>e.g., safety filters, resource limits</font>) or potentially be packaged by the framework if a tool fails critically. Check tool `FunctionResponse` content for typical tool-specific errors.

```python
// Example Error Event (conceptual)
{
  "author": "LLMAgent",
  "invocation_id": "e-err...",
  "content": null,
  "error_code": "SAFETY_FILTER_TRIGGERED",
  "error_message": "Response blocked due to safety settings.",
  "actions": {}
}
```

These details provide a more complete picture for advanced use cases involving tool authentication, state persistence scope, and error handling within the event stream.

### <b><font color='darkgreen'>Best Practices for Working with Events</font></b>
([source](https://google.github.io/adk-docs/events/#best-practices-for-working-with-events)) <b><font size='3ptx'>To use events effectively in your ADK applications:</font></b>

* <b><font size='3ptx'>Clear Authorship</font></b>: When building custom agents, ensure correct attribution for agent actions in the history. The framework generally handles authorship correctly for LLM/tool events.
    - Use `yield Event(author=self.name, ...)` in `BaseAgent` subclasses.
 
* <b><font size='3ptx'>Semantic Content & Actions</font></b>: Use `event.content` for the core message/data (<font color='brown'>text, function call/response</font>). Use `event.actions` specifically for signaling side effects (<font color='brown'>state/artifact deltas</font>) or control flow (<font color='brown'>transfer, escalate, skip_summarization</font>).
* <b><font size='3ptx'>Idempotency Awareness</font></b>: Understand that the SessionService is responsible for applying the state/artifact changes signaled in event.actions. While ADK services aim for consistency, <b>consider potential downstream effects if your application logic re-processes events</b>.

* <b><font size='3ptx'>Use `is_final_response()`</font></b>: Rely on this helper method in your application/UI layer to identify complete, user-facing text responses. Avoid manually replicating its logic.

* <b><font size='3ptx'>Leverage History</font></b>: The session's event list is your primary debugging tool. <b>Examine the sequence of authors, content, and actions to trace execution and diagnose issues</b>.

* <b><font size='3ptx'>Use Metadata</font></b>: Use `invocation_id` to correlate all events within a single user interaction. Use `event.id` to reference specific, unique occurrences.

Treating events as structured messages with clear purposes for their content and actions is key to building, debugging, and managing complex agent behaviors in ADK.